In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")
import tensorflow as tf
from sklearn.model_selection import *

In [2]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

In [3]:
train_input = train.drop(columns="label")
train_label = train["label"]
test_input = test.copy()

In [4]:
train_input

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41996,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41997,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41998,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
size_of_each_image = (int(np.sqrt(train_input.shape[1])),int(np.sqrt(train_input.shape[1])),1)
size_of_each_image

(28, 28, 1)

In [7]:
X_train,X_val,y_train,y_val = train_test_split(train_input,train_label,test_size=0.3,random_state=np.random.randint(0,1000))
X_test = test_input.copy()

In [8]:
X_train_size = tuple([X_train.shape[0]] + list(size_of_each_image))
X_val_size = tuple([X_val.shape[0]] + list(size_of_each_image))
X_test_size = tuple([X_test.shape[0]] + list(size_of_each_image))

In [16]:
X_train_size

(29400, 28, 28, 1)

In [10]:
X_train_1 = np.array(X_train)/255.0
X_val_1 = np.array(X_val)/255.0
X_test_1 = np.array(X_test)/255.0

In [11]:
X_train_2 = X_train_1.reshape(X_train_size)
X_val_2 = X_val_1.reshape(X_val_size)
X_test_2 = X_test_1.reshape(X_test_size)

y_train_2 = np.array(y_train)
y_val_2 = np.array(y_val)

In [12]:
X_train_2

array([[[[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        ...,

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]]],


       [[[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        ...,

        [[0.],
 

In [13]:
y_train_2

array([5, 6, 2, ..., 7, 6, 0])

In [14]:
test_size = np.unique(y_train).shape[0]
test_size

10

In [18]:
model = tf.keras.models.Sequential(layers=[
    tf.keras.layers.Resizing(height=64,width=64,input_shape=size_of_each_image),
    tf.keras.layers.Conv2D(filters=16,kernel_size=(3,3),activation="relu",input_shape=(64,64,1)),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
    tf.keras.layers.Conv2D(filters=32,kernel_size=(3,3),activation="relu"),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
    tf.keras.layers.Conv2D(filters=64,kernel_size=(3,3),activation="relu"),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
    tf.keras.layers.Conv2D(filters=128,kernel_size=(3,3),activation="relu"),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units=128,activation="relu"),
    tf.keras.layers.Dense(units=test_size,activation="softmax")
])

In [21]:
model.compile(
    optimizer = tf.keras.optimizers.legacy.RMSprop(),
    loss = tf.keras.losses.sparse_categorical_crossentropy,
    metrics = tf.keras.metrics.mean_squared_error
)

In [22]:
model.fit(
    x = X_train_2, y = y_train_2,
    batch_size = 32,
    epochs = 100,
    verbose = 1,
    callbacks = tf.keras.callbacks.EarlyStopping(patience=),
    validation_data = (X_val_2,y_val_2),
    shuffle = True,
    validation_batch_size = 32
)

Epoch 1/100
919/919 [==============================] - 23s 24ms/step - loss: 0.1880 - mean_squared_error: 27.5248 - val_loss: 0.0723 - val_mean_squared_error: 27.1151
Epoch 2/100
919/919 [==============================] - 23s 25ms/step - loss: 0.0538 - mean_squared_error: 27.5317 - val_loss: 0.0598 - val_mean_squared_error: 27.1160
Epoch 3/100
919/919 [==============================] - 22s 24ms/step - loss: 0.0366 - mean_squared_error: 27.5325 - val_loss: 0.0439 - val_mean_squared_error: 27.1167
Epoch 4/100
919/919 [==============================] - 22s 24ms/step - loss: 0.0253 - mean_squared_error: 27.5330 - val_loss: 0.0570 - val_mean_squared_error: 27.1163


In [25]:
cat_cross_entropy_loss, mean_squared_error_metric = model.evaluate(X_test_2)

875/875 [==============================] - 0s 334us/step - loss: 0.0000e+00 - mean_squared_error: 0.0000e+00


In [26]:
cat_cross_entropy_loss

0.0

In [27]:
mean_squared_error_metric

0.0

In [28]:
y_pred = model.predict(X_test_2)

875/875 [==============================] - 6s 7ms/step


In [31]:
y_pred

array([[2.5882940e-08, 2.1599870e-08, 9.9999905e-01, ..., 7.8283512e-07,
        2.3852522e-09, 3.2586911e-10],
       [9.9999607e-01, 2.4331473e-10, 3.4584470e-08, ..., 4.6397741e-08,
        1.9567385e-09, 1.8925585e-08],
       [3.7429547e-08, 8.6341984e-08, 9.3718018e-07, ..., 3.0317637e-05,
        7.9151097e-05, 9.9851328e-01],
       ...,
       [3.4679401e-15, 9.8047744e-11, 1.9713089e-11, ..., 6.1598704e-12,
        1.4169330e-09, 2.7541786e-14],
       [1.1963364e-07, 7.8862591e-08, 8.9259322e-07, ..., 3.5549125e-05,
        1.2521915e-07, 9.8534691e-01],
       [1.6152688e-12, 2.9679637e-10, 1.0000000e+00, ..., 1.2933765e-10,
        1.7683419e-10, 3.4001764e-15]], dtype=float32)

In [37]:
y_test = []
for i in y_pred:
    maxi = np.max(i)
    y_test.append(np.where(i == maxi)[0][0])
y_test = np.array(y_test)

In [42]:
test_df = pd.DataFrame(columns=["ImageId","Label"])
test_df["ImageId"] = range(1,len(y_test)+1)
test_df["Label"] = y_test

In [46]:
test_df.to_csv("amith_submission.csv",index=False)